In [ ]:
import random
import mysql.connector
from datetime import datetime, timedelta

# Input your MySQL username and password below

In [ ]:
username = 'coursera_user'
password = 'coursera_user'

In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user = username,
    password = password,
    database="GD_little_lemon_db"
)

In [ ]:
cursor = db.cursor()

# Step 1: Generate Orders from Bookings

In [ ]:
def check_and_insert_if_empty(table_name, insert_function):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    if count == 0:
        print(f"Table '{table_name}' is empty. Proceeding with data insertion.")
        insert_function()
    else:
        print(f"Table '{table_name}' is not empty. Skipping data insertion.")

# Step 2: Generate Order Items for Each Order

In [ ]:
def insert_orders():
    cursor.execute("SELECT id_booking FROM bookings")
    bookings = cursor.fetchall()

    for booking in bookings:
        id_booking = booking[0]
        date = datetime.now() - timedelta(days=random.randint(1, 90))  # Random date within last 3 months
        cursor.execute("INSERT INTO orders (date, fk_booking_id) VALUES (%s, %s)", (date, id_booking))
    db.commit()
    print(f"Inserted orders for {len(bookings)} bookings.\n")


check_and_insert_if_empty("orders", insert_orders)

# Step 3: Assign Staff to Bookings (Booking Staff Table)

In [ ]:
def insert_delivery_data():
    cursor.execute("SELECT id_order_items FROM order_items")
    order_items = cursor.fetchall()

    cursor.execute("SELECT id_staff FROM staff")
    staff = cursor.fetchall()

    statuses = ['ordered', 'preparing', 'ready for delivery', 'delivered']

    for item in order_items:
        id_order_item = item[0]
        fk_server = random.choice(staff)[0]

        # Insert each status with different timestamps
        for i, status in enumerate(statuses):
            timestamp = datetime.now() - timedelta(minutes=random.randint(1, 100) + (i * 10))  # Random time in the past 100 minutes, staggered by 10 mins
            cursor.execute(
                "INSERT INTO delivery (last_updated, status, fk_server, fk_order_items) VALUES (%s, %s, %s, %s)",
                (timestamp, status, fk_server, id_order_item)
            )

    db.commit()
    print(f"Inserted delivery records for {len(order_items)} order items with various statuses.\n")

# Finally, close the cursor and db connections

In [ ]:
cursor.close()
db.close()